<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U09/ANN09_Pretrained_Word_Embeddings_SOLUTION_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Verwendung vortrainierter Word-Embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Beschreibung:** Textklassifikation auf dem Newsgroup20-Datensatz unter Verwendung vortrainierter GloVe-Worteinbettungen.

## Setup

In [ ]:
import os
import torch
import pathlib
import tarfile
import zipfile
import logging
import numpy as np
import urllib.request
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F

from collections import Counter
from pytorch_lightning import Trainer

from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint

# Setup Standard Logging
logging.basicConfig(
    level="INFO",
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="[%H:%M:%S]",
)
logger = logging.getLogger(__name__)


## Vortrainierte Word-Embeddings

- In diesem Beispiel zeigen wir, wie man ein Textklassifizierungsmodell trainiert, das vortrainierte Worteinbettungen verwendet.
- Wir arbeiten mit dem Newsgroup20-Datensatz, einem Satz von 20.000 Nachrichten auf Messageboards die zu 20 verschiedenen Themenkategorien gehören.

Für die vortrainierten Worteinbettungen werden wir Folgendes verwenden
[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

## (a) Laden Sie den Newsgroup20 Datensatz herunter

- http://qwone.com/~jason/20Newsgroups/
- http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

Der 20 Newsgroups-Datensatz ist eine Sammlung von etwa 20.000 Newsgroup-Texten, die (fast) gleichmässig über 20 verschiedene Newsgroups verteilt sind. Sie wurde sie ursprünglich von Ken Lang gesammelt, wahrscheinlich für seinen Newsweeder: *Learning to filter netnews*, obwohl er diese Sammlung nicht ausdrücklich erwähnt. Die Sammlung von 20 Newsgroups ist zu einem beliebten Datensatz für Experimente mit Textanwendungen von maschinellen Lerntechniken geworden, wie z. B. Textklassifizierung und Textclustering.

**Organisation**
Die Daten sind in 20 verschiedene Newsgroups unterteilt, die jeweils einem bestimmten Thema entsprechen. Einige der Newsgroups sind sehr eng miteinander verwandt (z. B. comp.sys.ibm.pc.hardware / comp.sys.mac.hardware), während andere in keinem Zusammenhang stehen (z. B. misc.forsale / soc.religion.christian). Hier ist eine Liste der 20 Newsgroups, die (mehr oder weniger) nach Themen geordnet sind:

| computers                  | recreation              |  science    | politics |
|:---------------------------|:------------------------|:------------|:---------|
| `comp.graphics`            | `rec.autos`             | `sci.crypt` | `talk.politics.misc` |
| `comp.os.ms-windows.misc`  | `rec.motorcycles`       | `sci.electronics` | `talk.politics.misc` |
| `comp.sys.ibm.pc.hardware` | `rec.sport.baseball`    | `sci.med` | `talk.politics.guns` |
| `comp.sys.mac.hardware`    | `rec.sport.baseball`    | `sci.space` | `talk.politics.mideast` |
| `comp.windows.x`           | `rec.sport.baseball`    |            | `talk.religion.misc`|
|                            | `rec.sport.hockey`      |            |                     |

Ausserdem gibt es noch die Bereiche:

- `misc.forsale	`
- `alt.atheism`
- `soc.religion.christian`



Lädt den 20 Newsgroups-Datensatz herunter und extrahiert ihn ins angegebene Verzeichnis.

### Ablauf:
1. **URL festlegen**  
   `news20.tar.gz` wird von einer festen URL geladen.

2. **Verzeichnis anlegen**  
   `os.makedirs(data_dir)` erstellt Zielordner, falls nicht vorhanden.

3. **Download prüfen & durchführen**  
   Falls `news20.tar.gz` noch nicht existiert → Download mit `urllib.request.urlretrieve`.

4. **Entpacken**  
   Falls noch nicht extrahiert → Entpacken mit `tarfile.open(...).extractall(...)`.

5. **Pfad zurückgeben**  
   Gibt den Pfad zum entpackten 

In [ ]:
#####################################
# 0. Dataset herunterladen und extrahieren
#####################################


def download_and_extract_news20(data_dir):
    """
    Lädt den News20-Datensatz von der Online-Quelle herunter und extrahiert ihn.
    """
    url = "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz"
    os.makedirs(data_dir, exist_ok=True)
    tar_path = os.path.join(data_dir, "news20.tar.gz")
    if not os.path.exists(tar_path):
        logger.info("Downloading News20 dataset...")
        urllib.request.urlretrieve(url, tar_path)
    extract_path = os.path.join(data_dir, "20_newsgroup")
    if not os.path.exists(extract_path):
        logger.info("Extracting dataset...")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=data_dir)
    return extract_path


data_base_dir = "./news20_data"
data_dir = download_and_extract_news20(data_base_dir)
logger.info(f"Datensatzpfad: {data_dir}")


# (b) Daten laden und vorverarbeiten
Lädt den **20 Newsgroups**-Datensatz und bereitet ihn vor, indem die Header (ersten 10 Zeilen) jedes Dokuments entfernt werden.

### Ablauf:

1. **Pfad setzen**  
   `data_dir` wird in ein `Path`-Objekt umgewandelt für einfacheres Dateihandling.

2. **Klassennamen ermitteln**  
   Listet alle Unterordner (eine Klasse = ein Ordner), sortiert alphabetisch.

3. **Initialisierung von Datenlisten**  
   `samples` für die Texte, `labels` für die zugehörigen Klassen.

4. **Daten pro Klasse einlesen**  
   Für jeden Klassenordner:
   - Alle Dateien (außer versteckte) sammeln
   - Jede Datei:
     - Öffnen mit `latin-1` Encoding
     - Header (erste 10 Zeilen) entfernen
     - Restinhalt zu `samples` hinzufügen
     - Klassenindex zu `labels` hinzufügen

5. **Ausgabe**  
   Gibt eine Liste von Texten (`samples`), zugehörige Labels (`labels`) und die Klassennamen zurück.


In [ ]:
#####################################
# 1. Daten laden und Vorverarbeitung
#####################################


def load_news20_data(data_dir):
    """
    Liest den News20-Datensatz ein und entfernt die ersten 10 Zeilen (Header) jedes Dokuments.
    """
    data_dir = pathlib.Path(data_dir)
    class_names = sorted([d for d in os.listdir(data_dir) if not d.startswith(".")])
    samples = []
    labels = []
    for class_index, classname in enumerate(class_names):
        dirpath = data_dir / classname
        fnames = [
            f for f in os.listdir(dirpath) if not f.startswith(".") and os.path.isfile(dirpath / f)
        ]
        logger.info(f"Processing {classname}: {len(fnames)} Dateien gefunden")
        for fname in fnames:
            fpath = dirpath / fname
            with open(fpath, encoding="latin-1") as f:
                content = f.read()
                lines = content.split("\n")[10:]  # entferne Header
                content = "\n".join(lines)
                samples.append(content)
                labels.append(class_index)
    return samples, labels, class_names


samples, labels, class_names = load_news20_data(data_dir)
logger.info(f"Klassen: {class_names}")
logger.info(f"Anzahl Samples: {len(samples)}")


# (c) Trainings- und Validationsplit

In [ ]:
##########################################
# 2. Aufteilen in Trainings- und Validierungsdaten
##########################################

train_samples, val_samples, train_labels, val_labels = train_test_split(
    samples, labels, test_size=0.2, random_state=1337
)


# (d) Tokenizer und Vokabularaufbau
Dieser Abschnitt bereitet Textdaten für maschinelles Lernen vor, indem die Texte in **numerische Sequenzen** umgewandelt werden. Ziel: Die Texte in eine Form bringen, die ein neuronales Netz verarbeiten kann.

---

### Funktion: `tokenize(text)`
- Teilt einen Text in einzelne Wörter (Tokens).
- Alles wird in Kleinbuchstaben umgewandelt.
- **Beispiel:** `"Hello World"` → `["hello", "world"]`

---

### Funktion: `build_vocab(samples, max_tokens=20000)`
- Erstellt ein **Vokabular** der häufigsten Wörter.
- Nutzt `Counter`, um Wortfrequenzen zu zählen.
- Die häufigsten `max_tokens` Wörter erhalten fortlaufende Indizes:
  - `"<PAD>" = 0`: für Padding
  - `"<OOV>" = 1`: für unbekannte Wörter (Out-Of-Vocabulary)
- Gibt ein Dictionary `{wort: index}` zurück.

---

In [ ]:
############################################
# 3. Tokenizer und Vokabularaufbau
############################################


def tokenize(text):
    """Einfacher Tokenizer, der in Kleinbuchstaben in Wörter aufteilt."""
    return text.lower().split()


def build_vocab(samples, max_tokens=20000):
    """
    Baut ein Vokabular aus den häufigsten Wörtern auf.
    Reserviert Index 0 für Padding und 1 für unbekannte Wörter (OOV).
    """
    counter = Counter()
    for text in samples:
        tokens = tokenize(text)
        counter.update(tokens)
    most_common = counter.most_common(max_tokens)
    vocab = {"<PAD>": 0, "<OOV>": 1}
    for idx, (word, _) in enumerate(most_common, start=2):
        vocab[word] = idx
    return vocab


vocab = build_vocab(train_samples, max_tokens=20000)
logger.info(f"Groesse des Vokabulars: {len(vocab)}")


def encode_text(text, vocab, max_len=200):
    """
    Kodiert einen Text in eine Sequenz von Indizes.
    Schneidet auf max_len zu oder füllt mit Padding.
    """
    tokens = tokenize(text)
    encoded = [vocab.get(token, vocab["<OOV>"]) for token in tokens]
    if len(encoded) > max_len:
        encoded = encoded[:max_len]
    else:
        encoded += [vocab["<PAD>"]] * (max_len - len(encoded))
    return np.array(encoded, dtype=np.int64)


# (e) Dataset und Dataloader
Hier wird ein eigenes Dataset für PyTorch erstellt, um die vorbereiteten Texte und Labels effizient zu laden und zu batchen – ideal für Training und Validierung.

---

### Klasse: `NewsGroupDataset`
Ein benutzerdefiniertes Dataset, das mit `torch.utils.data.Dataset` kompatibel ist.

#### `__init__`
- Speichert:
  - `samples`: Liste von Texten
  - `labels`: zugehörige Klassenzuordnungen
  - `vocab`: Wörterbuch zur Umwandlung in Indizes
  - `max_len`: maximale Länge pro Text (für Padding/Truncation)

#### `__len__`
- Gibt die Anzahl der Samples zurück → wichtig für Batch-Iteration.

#### `__getitem__(idx)`
- Holt das `idx`-te Text-Label-Paar.
- Wandelt den Text mit `encode_text` in eine Zahlen-Sequenz um.
- Gibt ein Tupel aus `(encoded_tensor, label_tensor)` zurück – beide als `torch.tensor`.

---

In [ ]:
##############################################
# 4. Dataset und DataLoader
##############################################


class NewsGroupDataset(Dataset):
    def __init__(self, samples, labels, vocab, max_len=200):
        self.samples = samples
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        text = self.samples[idx]
        label = self.labels[idx]
        encoded = encode_text(text, self.vocab, self.max_len)
        return torch.tensor(encoded, dtype=torch.long), torch.tensor(label, dtype=torch.long)


batch_size = 128
train_dataset = NewsGroupDataset(train_samples, train_labels, vocab, max_len=200)
val_dataset = NewsGroupDataset(val_samples, val_labels, vocab, max_len=200)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# (f) GloVe Einbettungen laden
In diesem Abschnitt werden **vortrainierte GloVe-Wortvektoren** heruntergeladen, eingelesen und in eine **Embedding-Matrix** für das Modell überführt.

---

### Funktion: `download_glove_embeddings(...)`
- Lädt das GloVe-Zip-Archiv von der offiziellen Stanford-Seite.
- Entpackt es im Zielverzeichnis, falls es dort noch nicht vorliegt.
- Gibt den Pfad zur gewünschten `.txt`-Datei zurück (z. B. `glove.6B.100d.txt`).

---

### Funktion: `get_embedding_dim_from_filename(...)`
- Extrahiert die Vektor-Dimension (z. B. `100d`) aus dem Dateinamen.
- Praktisch, um `embedding_dim` automatisch zu setzen.

---

### Funktion: `load_glove_embeddings(...)`
- Liest die GloVe-Datei Zeile für Zeile ein.
- Jede Zeile enthält ein Wort + Vektorwerte → wird als `np.array` gespeichert.
- Ergebnis: Dictionary `{wort: vektor}`

---

In [ ]:
#####################################################
# 5. Vortrainierte GloVe-Einbettungen laden
#####################################################


def load_glove_embeddings(glove_file_path):
    """
    Liest GloVe-Einbettungen aus der Datei ein.
    """
    embeddings_index = {}
    with open(glove_file_path, encoding="utf8", errors="ignore") as f:
        for line in f:
            values = line.split()
            if len(values) < 2:
                continue
            word = values[0]
            try:
                vector = np.asarray(values[1:], dtype=np.float32)
                embeddings_index[word] = vector
            except ValueError:
                logger.warning("Skipping line due to ValueError.")
                continue
    return embeddings_index


def download_glove_embeddings(glove_dir, glove_file="glove.6B.100d.txt"):
    """
    Lädt die GloVe-Einbettungen herunter und entpackt sie, falls noch nicht vorhanden.
    """
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    os.makedirs(glove_dir, exist_ok=True)
    zip_path = os.path.join(glove_dir, "glove.6B.zip")
    glove_file_path = os.path.join(glove_dir, glove_file)

    if not os.path.exists(glove_file_path):
        if not os.path.exists(zip_path):
            logger.info("Downloading GloVe embeddings...")
            urllib.request.urlretrieve(url, zip_path)
        logger.info("Extracting GloVe embeddings...")
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(glove_dir)
    return glove_file_path


import re


def get_embedding_dim_from_filename(glove_file):
    match = re.search(r"\.(\d+)d\.txt$", glove_file)
    if match:
        return int(match.group(1))
    raise ValueError(f"Ungültiger GloVe-Dateiname: {glove_file}")


glove_dir = "./glove_data"
glove_file = "glove.6B.100d.txt"  # oder "glove.6B.300d.txt", "glove.6B.50d.txt", etc.
glove_file_path = download_glove_embeddings(glove_dir, glove_file=glove_file)

# Automatisch korrekte Dimensionalität setzen
embedding_dim = get_embedding_dim_from_filename(glove_file)
logger.info(f"Verwende embedding_dim = {embedding_dim}")

glove_embeddings = load_glove_embeddings(glove_file_path)

num_tokens = len(vocab)
embedding_matrix = np.zeros((num_tokens, embedding_dim), dtype=np.float32)

hits, misses = 0, 0
for word, i in vocab.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

logger.info(f"Converted {hits} words ({misses} misses)")


# (g) PyTorch Modell aufstellen
Dieses Modell ist ein **Convolutional Neural Network (CNN)** zur **Textklassifikation**, basierend auf GloVe-Embeddings.

---

### Architekturüberblick:

#### `__init__`
- **Embedding-Schicht**  
  Vortrainierte GloVe-Vektoren, nicht trainierbar (`requires_grad=False`)

- **Convolutional Layers**  
  Drei 1D-Convs mit ReLU + Max-Pooling zur Extraktion lokaler Textmuster.

- **Pooling & Dense Layer**  
  - Globales Max-Pooling → reduziert Sequenz auf festen Vektor.
  - `fc1` + ReLU + Dropout → Regularisierung
  - `fc2` → Finale Klassenvorhersage

---

### `forward(x)`
- Input: `(batch_size, sequence_length)`
- Output: Logits für jede Klasse → `(batch_size, num_classes)`

---

### `training_step(...)` & `validation_step(...)`
- Berechnen **Cross-Entropy Loss** und **Accuracy**
- Loggen Metriken für jede Epoche
- Speichern zusätzlich `loss` & `acc` in Listen (z. B. für spätere Visualisierung)

---

### `configure_optimizers()`
- Verwendet **RMSprop** als Optimierer mit dem vorgegebenen Lernrate-Wert.

---

### `predict_text(...)`
- Nutzt das Modell, um neue Texte zu kodieren und Klassifikationswahrscheinlichkeiten (via `softmax`) zurückzugeben.

---

### Ziel:
`TextCNN` erkennt Textmuster über Convolutional Filters und eignet sich besonders gut für **Textklassifikation mit fixierter Eingabelänge** (z. B. bei Dokumenten oder Tweets).

In [ ]:
########################################################
# 6. Modell in PyTorch Lightning: TextCNN
########################################################


class TextCNN(pl.LightningModule):
    def __init__(self, embedding_matrix, num_classes, dropout=0.5, lr=1e-3):
        super(TextCNN, self).__init__()
        num_tokens, embedding_dim = embedding_matrix.shape
        # Embedding-Schicht: vortrainierte Einbettungen, die fixiert sind.
        self.embedding = nn.Embedding(num_tokens, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix), requires_grad=False)
        # Convolutional Layers
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=128, kernel_size=5)
        self.pool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5)
        self.pool2 = nn.MaxPool1d(kernel_size=5)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5)
        # Dense Layers
        self.fc1 = nn.Linear(128, 128)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(128, num_classes)
        self.lr = lr

        # loss Kurven
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []

    def forward(self, x):
        # x: (batch_size, sequence_length)
        x = self.embedding(x)  # (batch_size, sequence_length, embedding_dim)
        x = x.transpose(1, 2)  # (batch_size, embedding_dim, sequence_length)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = torch.max(x, dim=2)[0]  # Globales Max-Pooling
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        logits = self.fc2(x)
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        # loss Kurven
        self.train_loss.append(loss.item())
        self.train_acc.append(acc.item())
        self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log("train_acc", acc, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        # loss Kurven
        self.val_loss.append(loss.item())
        self.val_acc.append(acc.item())
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=self.lr)
        return optimizer

    def predict_text(self, texts, vocab, max_len=200):
        """
        Kodiert Texte und gibt Vorhersagen als Wahrscheinlichkeiten zurück.
        """
        self.eval()
        encoded = np.array([encode_text(text, vocab, max_len) for text in texts])
        x = torch.tensor(encoded, dtype=torch.long).to(self.device)
        with torch.no_grad():
            logits = self(x)
            probs = F.softmax(logits, dim=1)
        return probs.cpu().numpy()


# (h) Training und Inferenz
In diesem Codeabschnitt wird ein TextCNN-Modell trainiert:

- Die Trainingsumgebung wird mit Checkpointing und Logging eingerichtet.

- Das Modell speichert automatisch das beste Ergebnis auf Basis der Validierungsgenauigkeit.

- Die Trainingsstatistiken werden in einer CSV-Datei protokolliert.

In [ ]:
#########################################
# 7. Training und Inferenz
#########################################
np.Inf = np.inf
num_classes = len(class_names)
model = TextCNN(embedding_matrix, num_classes)


# Callback für das Speichern des besten Modells basierend auf der Validierungsgenauigkeit
checkpoint_callback = ModelCheckpoint(
    monitor="val_acc",  # Überwache die Validierungsgenauigkeit
    mode="max",  # Maximierung der Genauigkeit
    save_top_k=1,  # Speichere nur das beste Modell
    filename="best-checkpoint",  # Dateiname des besten Modells
    verbose=True,
)

# CSV Logger hinzufügen
csv_logger = pl.loggers.CSVLogger(save_dir="logs/", name="textcnn_logs")

# Trainer mit automatischer Geräteauswahl und CSV Logger
trainer = Trainer(
    max_epochs=15,
    accelerator="auto",
    devices="auto",
    callbacks=[checkpoint_callback],
    logger=csv_logger,
)
trainer.fit(model, train_loader, val_loader)


In [ ]:
print(csv_logger.log_dir)


In [ ]:
import os
import pandas as pd


def find_latest_metrics_path(base_log_dir):
    version_dirs = [d for d in os.listdir(base_log_dir) if d.startswith("version_")]
    if not version_dirs:
        raise FileNotFoundError("Keine version_x Ordner im Log-Verzeichnis gefunden.")

    # Nach Versionsnummer sortieren
    version_dirs.sort(key=lambda x: int(x.split("_")[1]))
    latest_version = version_dirs[-1]

    return os.path.join(base_log_dir, latest_version, "metrics.csv")


# Hauptverzeichnis für Logs
base_log_dir = os.path.join("logs", "textcnn_logs")
metrics_path = find_latest_metrics_path(base_log_dir)

print("Neueste metrics.csv gefunden unter:", metrics_path)

# DataFrame vorbereiten
df = pd.read_csv(metrics_path)
df_train = df[["epoch", "train_loss", "train_acc"]].dropna().copy()
df_val = df[["epoch", "val_loss", "val_acc"]].dropna().copy()

# Merge auf 'epoch'
df_merged = pd.merge(df_train, df_val, on="epoch", suffixes=("_train", "_val"))

df_merged


In [ ]:
# plotte df_merged mit seaborn
import seaborn as sns
import matplotlib.pyplot as plt

# Setze den Stil für die Plots
sns.set(style="whitegrid")
# Erstelle eine Figur mit zwei Subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
# Plot für den Verlust
sns.lineplot(data=df_merged, x="epoch", y="train_loss", ax=ax1, label="Trainingsverlust")
sns.lineplot(data=df_merged, x="epoch", y="val_loss", ax=ax1, label="Validierungsverlust")
ax1.set_title("Trainings- und Validierungsverlust")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Verlust")
ax1.legend()
# Plot für die Genauigkeit
sns.lineplot(data=df_merged, x="epoch", y="train_acc", ax=ax2, label="Trainingsgenauigkeit")
sns.lineplot(data=df_merged, x="epoch", y="val_acc", ax=ax2, label="Validierungsgenauigkeit")
ax2.set_title("Trainings- und Validierungsgenauigkeit")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Genauigkeit")
ax2.legend()
plt.tight_layout()
plt.show()



1. **Beispieltexte definieren**  
   Eine Liste `sample_texts` mit fünf Texten wird angelegt, die später klassifiziert werden sollen.

2. **Bestes Modell laden**  
   - `best_model_path` holt den Pfad zum aktuell besten Checkpoint aus `checkpoint_callback`.  
   - Mit `TextCNN.load_from_checkpoint(...)` wird das gespeicherte Modell geladen (inkl. `embedding_matrix` und `num_classes`).

3. **Evaluierungsmodus aktivieren**  
   Durch `best_model.eval()` schaltet das Modell in den Inferenz‑Modus (z. B. deaktiviert Dropout).

4. **Klassenvorhersagen berechnen**  
   - `model.predict_text(texts=sample_texts, vocab=vocab)` liefert für jeden Text die Wahrscheinlichkeitsverteilung über die Klassen.  
   - `probs.argmax(axis=1)` wählt jeweils die Klasse mit der höchsten Wahrscheinlichkeit aus.

5. **Ergebnis**  
   `predicted_classes` enthält die vorhergesagten Klassenindices für die fünf Beispieltexte.  

In [ ]:
# Beispiel-Inferenz
sample_texts = [
    "this message is about computer graphics and 3D modeling",
    "I firmly believe in God.",
    "I firmly believe in God and all the angels.",
    "it's time to make peace in the Far East. War is bad and should be avoided.",
    "the new Volvo is excellent. It has drive by wire and automatic gear control.",
]


best_model_path = checkpoint_callback.best_model_path
logger.info(f"Lade bestes Modell von: {best_model_path}")

best_model = TextCNN.load_from_checkpoint(
    best_model_path, embedding_matrix=embedding_matrix, num_classes=num_classes
)
best_model.eval()
probs = model.predict_text(texts=sample_texts, vocab=vocab)
predicted_classes = probs.argmax(axis=1)


In [ ]:
# Ausgabe der Vorhersagen in normalem Logging-Format
logger.info("Beispiel-Inferenz:")
logger.info(f"{'Text':<80} | {'Predicted Class':<15}")
logger.info("-" * 100)
for text, pred in zip(sample_texts, predicted_classes):
    logger.info(f"{text:<80} | {class_names[pred]:<15}")


In [ ]:
# Berechnung und Ausgabe der Accuracy
y_pred = model.predict_text(texts=val_samples, vocab=vocab)
y_label_pred = [np.argmax(prob) for prob in y_pred]
acc = accuracy_score(y_true=y_label_pred, y_pred=val_labels)
logger.info("----- Evaluation Summary -----")
logger.info(f"Accuracy: {acc:.2%}")
